In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("Cleaned1.csv")
df.head()

,Country,Balance,Product,Value,Unit,Month,Date
0,Australia,Net Electricity Production,Electricity,23130.2764,GWh,Mar,24
1,Australia,Net Electricity Production,Total Combustible Fuels,14353.8714,GWh,Mar,24
2,Australia,Net Electricity Production,"Coal, Peat and Manufactured Gases",10304.7825,GWh,Mar,24
3,Australia,Net Electricity Production,Oil and Petroleum Products,330.5351,GWh,Mar,24
4,Australia,Net Electricity Production,Natural Gas,3492.4621,GWh,Mar,24


In [3]:
# split dataframe...
df1 = df[df['Balance'] == 'Net Electricity Production']

In [4]:
df2 = df[df['Balance'] != 'Net Electricity Production']

In [5]:
dff1 = df1 # Copying dataframe which having 'Balance' == 'Net Electricity Production'
dff2 = df2 # Copying dataframe which having 'Balance' != 'Net Electricity Production'

# For self security  (-;

In [6]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [7]:
combined_countries = pd.concat([dff1['Country'], dff2['Country']]).unique()
combined_balances = pd.concat([dff1['Balance'], dff2['Balance']]).unique()
combined_products = pd.concat([dff1['Product'], dff2['Product']]).unique()
combined_months = pd.concat([dff1['Month'], dff2['Month']]).unique()

In [8]:
country_encoder = LabelEncoder().fit(combined_countries)
balance_encoder = LabelEncoder().fit(combined_balances)
product_encoder = LabelEncoder().fit(combined_products)
month_encoder = LabelEncoder().fit(combined_months)

In [9]:
dff1 = dff1.drop(['Unit'], axis=1)
dff1['Country'] = country_encoder.transform(dff1['Country'])
dff1['Balance'] = balance_encoder.transform(dff1['Balance'])
dff1['Product'] = product_encoder.transform(dff1['Product'])
dff1['Month'] = month_encoder.transform(dff1['Month'])


In [10]:
# Transform dff2
dff2 = dff2.drop(['Unit'], axis=1)
dff2['Country'] = country_encoder.transform(dff2['Country'])
dff2['Balance'] = balance_encoder.transform(dff2['Balance'])
dff2['Product'] = product_encoder.transform(dff2['Product'])
dff2['Month'] = month_encoder.transform(dff2['Month'])

In [11]:
# Save the encoders for future use
joblib.dump(country_encoder, 'country_encoder2.pkl')
joblib.dump(balance_encoder, 'balance_encoder2.pkl')
joblib.dump(product_encoder, 'product_encoder2.pkl')
joblib.dump(month_encoder, 'month_encoder2.pkl')

['month_encoder2.pkl']

In [14]:
# Model training for df1
x = dff1.drop(columns=['Value'], axis=1)
y = dff1['Value']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=3)

# Standard Scaler Transform
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

model1 = RandomForestRegressor(n_estimators=100)
model1.fit(x_train, y_train)
joblib.dump(model1, 'model_3.pkl')

['model_3.pkl']

In [15]:
# Model training for df2
p = dff2.drop(columns=['Value'], axis=1)
q = dff2['Value']
p_train, p_test, q_train, q_test = train_test_split(p, q, test_size=0.3, random_state=3)

# Standard Scaler Transform
p_train = sc.fit_transform(p_train)
p_test = sc.transform(p_test)

model2 = RandomForestRegressor(n_estimators=100)
model2.fit(p_train, q_train)
joblib.dump(model2, 'model_4.pkl')

['model_4.pkl']

In [18]:
# Prediction Example
# Example input
input_data = [['Austria', 'Total Exports', 'Natural Gas', 'Mar', 24]]

# Load the encoders
country_encoder = joblib.load('country_encoder2.pkl')
balance_encoder = joblib.load('balance_encoder2.pkl')
product_encoder = joblib.load('product_encoder2.pkl')
month_encoder = joblib.load('month_encoder2.pkl')

# Create a DataFrame for the input data
input_df = pd.DataFrame(input_data, columns=['Country', 'Balance', 'Product', 'Month', 'Data'])

# Encode the input data
input_df['Country'] = country_encoder.transform(input_df['Country'])
input_df['Balance'] = balance_encoder.transform(input_df['Balance'])
input_df['Product'] = product_encoder.transform(input_df['Product'])
input_df['Month'] = month_encoder.transform(input_df['Month'])

# Extract the features
input_features = input_df[['Country', 'Balance', 'Product', 'Month', 'Data']].values

# Load the trained model
model = joblib.load('model_4.pkl')  # Use model_2.pkl if appropriate

# Make a prediction
predicted_price = model.predict(input_features)
print(f"The predicted price is: {predicted_price[0]}")

The predicted price is: 2725.738648


In [20]:
input_data = [['Austria', 'Net Electricity Production', 'Natural Gas', 'Mar', 24]]

# Load the encoders
country_encoder = joblib.load('country_encoder2.pkl')
balance_encoder = joblib.load('balance_encoder2.pkl')
product_encoder = joblib.load('product_encoder2.pkl')
month_encoder = joblib.load('month_encoder2.pkl')

# Create a DataFrame for the input data
input_df = pd.DataFrame(input_data, columns=['Country', 'Balance', 'Product', 'Month', 'Data'])

# Encode the input data
input_df['Country'] = country_encoder.transform(input_df['Country'])
input_df['Balance'] = balance_encoder.transform(input_df['Balance'])
input_df['Product'] = product_encoder.transform(input_df['Product'])
input_df['Month'] = month_encoder.transform(input_df['Month'])

# Extract the features
input_features = input_df[['Country', 'Balance', 'Product', 'Month', 'Data']].values

# Load the trained model
model = joblib.load('model_3.pkl')  # Use model_2.pkl if appropriate

# Make a prediction
predicted_price = model.predict(input_features)
print(f"The predicted price is: {predicted_price[0]}")

The predicted price is: 28792.899015000006
